# Immigration, Deportation, and Discrimination: Hispanic Political Opinion Since the Election of Donald Trump


In [1]:
#Packages
library(InformationValue)
library(stringr)
library(data.table)
library(scales)
library(stargazer)
library(multiwayvcov)
library(lmtest)
library(het.test)
library(sandwich)
library(margins)
library(maxLik)
library(pscl)
library(BaylorEdPsych)



Please cite as: 


 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: vars

Loading required package: MASS

Loading required package: strucchange

Loading required package: sandwich


Attaching package: ‘strucchange’


The following object is masked from ‘package:stringr’:

    boundary


Loading required package: urca

Loading required package: miscTools


Please cite the 'maxLik' package as:
Henningsen, Arne and Toomet, Ott (2011). maxLik: A package for maximum likelihood estimation in R. Computational Statistics 26(3), 443-458. DOI 10.1007/s00180-010-0217-1.

If you have questions, suggestions, or comments regarding the 'maxLik' package, please use a forum or 'tracker' at maxLik's R-Forge site:
https:

Survey Questions:

**Q4** What country were you born in?

**Q5** How long have you lived in the United States (including Puerto Rico)? 

**Q8** Mark the statement that you MOST agree with: 
> Generally, illegal immigrants should NOT be deported (1)<br>
Generally, illegal immigrants should be deported (2)

**Q11** In the 2016 Presidential Election, who did you vote for? 
> Donald Trump (1)<br>
Hillary Clinton (2)<br>
Other (3)<br>
Did not vote (4)

**Q17** On a scale from 1-7, how worried are you about the following situations? <br>
(1) Not worried, (4) Somewhat worried, (7) Very worried 
> Myself or close family members being deported (1)         
Friends or close acquaintances being deported (2)<br>
The separation of families due to deportation (3)<br>
The economy and/or workforce being negatively impacted due to deportations (4)

**Q18** Do you personally know a Latino(a)/Hispanic who is undocumented and currently living in the United States? (Including yourself)

If Yes is selected for Q18: <br>
**Q19** To what degree do illegal immigrants you know well (including yourself) worry about participating in the following situations for fear of deportation? (Sliding Scale) <br>
(1) Do not worry, (4) Worry somewhat, (7) Worry very much 
> Applying for scholarships or loans for education (1) <br>
Health care and/or medical assistance (2) <br>
Jobs (3) <br>
Government welfare or assistance (4) <br>
Reporting crimes (5) <br>
Attending public schools (6)

**Q20** Since the campaign and election of Donald Trump, do you feel levels of discrimination towards Latinos/Hispanics have increased? (Sliding Scale) <br>
(1) Not at all, (4) Somewhat, (7) Very much

**Q23** How old are you?

**Q24** What is your gender?


In [2]:
#Load data
data <- read.csv("SurveyData.csv", header = TRUE)

In [3]:
#Define Variables

#Split by region
US <- c('USA', 'Puerto Rico')
Central <- c('Costa Rica', 'El Salvador', 'Guatemala', 'Honduras', 'Mexico', 'Nicaragua', 'Panama')
South <- c('Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Paraguay', 'Peru', 'Uruguay', 'Venezuela')
data$region <- as.factor(ifelse(data$Q4 %in% US, 'US/Puerto Rico', 
                         ifelse(data$Q4 %in% Central, 'Central America', 
                         ifelse(data$Q4 %in% South, 'South America',
                         ifelse(data$Q4 == 'Cuba', 'Cuba', 
                         ifelse(data$Q4 == 'Other', 'Other', NA))))))

#Group Ages
data$age <- as.factor(ifelse(data$Q23 == '18-24'|data$Q23 == '25-30', '18-30', 
                      ifelse(data$Q23 == '31-40'|data$Q23 == '41-50', '31-50', 
                      ifelse(data$Q23 == '51-60'|data$Q23 == '61+', '51+', NA))))

#Group Years in US
data$yearsInUS <- as.factor(ifelse(data$Q5 == 1, '<5', 
                            ifelse(data$Q5 == 2, '6-10',
                            ifelse(data$Q5 > 2, '11+', NA))))



In [4]:
#Demographic Characteristics
print("Demographic Characteristics of Sample")

#change names of Q24 and Q18 to gender and knowUndocumented
colnames(data)[colnames(data)=="Q24"] <- "gender"
colnames(data)[colnames(data)=="Q18"] <- "knowUndocumented"

#total observations
total = nrow(data)

#function to print number and percentages for each characteristic
characteristics <- function(x, word) {
    t <- table(x)
    total1 <- sum(t)
    print(word)
    print(t)
    print(round(100*t/total1,1))
    print('NAs')
    print(total - total1)
}

#use function for each characteristic 
for (x in list(list(data$Language, 'Language'), 
               list(data$gender, 'Gender'),
               list(data$age, 'Age'),
               list(data$region, 'Region'),
               list(data$yearsInUS, 'Years In US'),
               list(data$knowUndocumented, 'Know Undocumented'))) {
    characteristics(x[1], x[2])
}

[1] "Demographic Characteristics of Sample"
[[1]]
[1] "Language"

x
 EN  ES 
925 155 
x
  EN   ES 
85.6 14.4 
[1] "NAs"
[1] 0
[[1]]
[1] "Gender"

x
Female   Male 
   612    452 
x
Female   Male 
  57.5   42.5 
[1] "NAs"
[1] 16
[[1]]
[1] "Age"

x
18-30 31-50   51+ 
  416   429   222 
x
18-30 31-50   51+ 
 39.0  40.2  20.8 
[1] "NAs"
[1] 13
[[1]]
[1] "Region"

x
Central America            Cuba           Other   South America  US/Puerto Rico 
            116              38              35              77             685 
x
Central America            Cuba           Other   South America  US/Puerto Rico 
           12.2             4.0             3.7             8.1            72.0 
[1] "NAs"
[1] 129
[[1]]
[1] "Years In US"

x
  <5  11+ 6-10 
  94  907   66 
x
  <5  11+ 6-10 
 8.8 85.0  6.2 
[1] "NAs"
[1] 13
[[1]]
[1] "Know Undocumented"

x
  0   1 
475 590 
x
   0    1 
44.6 55.4 
[1] "NAs"
[1] 15


In [5]:
#Chi-squared tests for Q8

#function to print out percentages and perform chi-squared test for a given characteristic and question
chiSquared <- function(x, y, word) {
    t <- as.data.frame.matrix(table(x,y))
    percents <- t
    totals <- apply(t,1,sum)
    for(i in 1:nrow(t)) {
        percents[i,] <- round(100*t[i,]/totals[i], 1)
    }
    print(word)
    print(percents)
    chisq.test(t)
}

#Q8
print('Do You Agree With the Statement That Generally, All Illegal Immigrants Should Be Deported?')

#use function for each characteristic of interest on Q8
chiSquared(data$age, data$Q8, 'Age')
chiSquared(data$gender, data$Q8, 'Gender')
chiSquared(data$Language, data$Q8, 'Language')
chiSquared(data$yearsInUS, data$Q8, 'yearsInUS')
chiSquared(data$knowUndocumented, data$Q8, 'knowUndocumented')

#totals for Q8
print('Total')
print(100*table(data$Q8)/sum(table(data$Q8)))

#number of responses for Q8
print('N')
print(nrow(data[!is.na(data$Q8),]))

[1] "Do You Agree With the Statement That Generally, All Illegal Immigrants Should Be Deported?"
[1] "Age"
         0    1
18-30 73.7 26.3
31-50 66.1 33.9
51+   50.5 49.5



	Pearson's Chi-squared test

data:  t
X-squared = 34.072, df = 2, p-value = 3.993e-08


[1] "Gender"
          0    1
Female 69.2 30.8
Male   60.9 39.1



	Pearson's Chi-squared test with Yates' continuity correction

data:  t
X-squared = 7.3779, df = 1, p-value = 0.006603


[1] "Language"
      0    1
EN 63.1 36.9
ES 81.9 18.1


...

[1] "yearsInUS"
        0    1
<5   66.7 33.3
11+  64.9 35.1
6-10 74.6 25.4



	Pearson's Chi-squared test

data:  t
X-squared = 2.4969, df = 2, p-value = 0.2869


[1] "knowUndocumented"
     0    1
0 58.1 41.9
1 71.8 28.2



	Pearson's Chi-squared test with Yates' continuity correction

data:  t
X-squared = 21.043, df = 1, p-value = 4.49e-06


[1] "Total"

      0       1 
65.6816 34.3184 
[1] "N"
[1] 1049


In [6]:
#explanatory variables

#dummy variables for different characteristics 
data$trump <- ifelse(data$Q11 == 'Trump', 1, 0)
data$clinton <- ifelse(data$Q11 == 'Clinton', 1, 0)
data$young <- ifelse(data$age == '18-30', 1, 0)
data$middle <- ifelse(data$age == '31-50', 1, 0)
data$old <- ifelse(data$age == '51+', 1, 0)
data$yearsUS_5 <- ifelse(data$yearsInUS == '<5', 1, 0)
data$yearsUS_6_10 <- ifelse(data$yearsInUS == '6-10', 1, 0)

#interaction terms
data$trump_old <- ifelse(data$trump == 1 & data$old == 1, 1, 0)
data$clinton_old <- ifelse(data$clinton == 1 & data$old == 1, 1, 0)
data$clinton_knowUndocumented <- ifelse(data$clinton == 1 & data$knowUndocumented == 1, 1, 0)


In [7]:
#response variables

#split Q20 into 3 categories
data$discrimination <- as.factor(ifelse(data$Q20 <= 3, 'not at all', 
                      ifelse(data$Q20 < 5 & data$Q20 > 3, 'somewhat', 
                      ifelse(data$Q20 >= 5, 'very much', NA))))

#convert 7-point scale questions into binary variables
data$veryWorriedFriends <- ifelse(data$Q17_2 >= 5, 1, 0)
data$veryWorriedFamilies <- ifelse(data$Q17_3 >= 5, 1, 0)
data$veryWorriedEconomy <- ifelse(data$Q17_4 >= 5, 1, 0)
data$discriminationIncreased <- ifelse(data$discrimination == 'very much', 1, 0)
data$discriminationNotIncreased <- ifelse(data$discrimination == 'not at all', 1, 0)


In [8]:
#Logit Model 1 for Q8
print('Logit Model 1 for whether or not participants agreed with the statement that generally, all illegal immigrants should be deported (marginal effect)')

#total observations for model 
print('N')
nrow(data[!is.na(data$Q8) & 
                 !is.na(data$Language) &
                 !is.na(data$knowUndocumented) & 
                 !is.na(data$age) &
                 !is.na(data$Q11), ])

#model with spanish, 18-30 years old, know undocumented, voted for Trump, and voted for Clinton
model1 <-glm(Q8 ~ Language + young + knowUndocumented + trump + clinton,
    family = binomial(link = "logit"),
    data = data)

#marginal effect
summary(margins(model1))

#percent correctly predicted and Mckelvey-Zavoina pseudo R2 for model
PseudoR2(model1)




[1] "Logit Model 1 for whether or not participants agreed with the statement that generally, all illegal immigrants should be deported (marginal effect)"
[1] "N"


[1] 1043

factor,AME,SE,z,p,lower,upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
clinton,-0.1142169,0.02908291,-3.927284,8.591038e-05,-0.1712183,-0.05721540
knowUndocumented,-0.1071845,0.02578978,-4.156085,3.237476e-05,-0.1577316,-0.05663748
LanguageES,-0.1764906,0.03328375,-5.302606,1.141612e-07,-0.2417256,-0.11125567
trump,0.3256457,0.03061113,10.638146,1.980312e-26,0.2656490,0.38564239
young,-0.1075557,0.02726878,-3.944279,8.004027e-05,-0.1610015,-0.05410987


McFadden     Adj.McFadden        Cox.Snell       Nagelkerke 
       0.1697276        0.1592824        0.1959652        0.2709058 
McKelvey.Zavoina           Effron            Count        Adj.Count 
       0.2640316        0.2202587        0.7526366        0.2773109 
             AIC    Corrected.AIC 
    1124.8415707     1124.9226518

In [9]:
#Logit Model 2 for Q8
print('Logit Model 2 for whether or not participants agreed with the statement that generally, all illegal immigrants should be deported (marginal effect)')

#total observations for model 
print('N')
nrow(data[!is.na(data$Q8) & 
                 !is.na(data$Language) &
                 !is.na(data$knowUndocumented) & 
                 !is.na(data$age) &
                 !is.na(data$Q11), ])

#model with Spanish, 18-30 years old, know undocumented, voted for Trump, voted for Clinton, Clinton*know undocumented, Trump*old
model2 <-glm(Q8 ~ Language + young + knowUndocumented + trump + clinton + clinton_knowUndocumented + trump_old,
    family = binomial(link = "logit"),
    data = data)

#marginal effect
summary(margins(model2))

#percent correctly predicted and Mckelvey-Zavoina pseudo R2 for model
PseudoR2(model2)



[1] "Logit Model 2 for whether or not participants agreed with the statement that generally, all illegal immigrants should be deported (marginal effect)"
[1] "N"


[1] 1043

factor,AME,SE,z,p,lower,upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
clinton,-0.18871167,0.03839021,-4.915619,8.850228e-07,-0.26395511,-0.11346824
clinton_knowUndocumented,0.16485020,0.05246660,3.142003,1.677963e-03,0.06201757,0.26768284
knowUndocumented,-0.16732434,0.03267081,-5.121524,3.030759e-07,-0.23135795,-0.10329073
LanguageES,-0.15914013,0.03254700,-4.889549,1.010672e-06,-0.22293107,-0.09534919
trump,0.28041510,0.03558781,7.879526,3.286244e-15,0.21066427,0.35016592
trump_old,0.26544238,0.09696579,2.737485,6.191099e-03,0.07539292,0.45549185
young,-0.09050644,0.02714166,-3.334595,8.542384e-04,-0.14370312,-0.03730976


McFadden     Adj.McFadden        Cox.Snell       Nagelkerke 
       0.1850149        0.1715854        0.2116065        0.2925287 
McKelvey.Zavoina           Effron            Count        Adj.Count 
       0.3134652        0.2361950        0.7497603        0.2689076 
             AIC    Corrected.AIC 
    1108.3515779     1108.4908429

In [10]:
#Logit Model for Q17_2
print("Logit Model for Whether or Not Participants Were Very Worried About Friends or Close Acquaintances Being Deported (Marginal Effect)")

#total observations for model 
print('N')
nrow(data[!is.na(data$Q17_2) & 
                 !is.na(data$Language) &
                 !is.na(data$knowUndocumented) & 
                 !is.na(data$Q11) &
                 !is.na(data$yearsInUS) &
                 !is.na(data$age), ])

#model with Spanish, know undocumented, voted for Clinton, lived in US fewer than 5 years, lived in US between 6 and 10 years, and 51+ years old
model <-glm(veryWorriedFriends ~ Language + knowUndocumented + clinton + yearsInUS + old,
    family = binomial(link = "logit"),
    data = data)

#marginal effect
summary(margins(model))

#percent correctly predicted and Mckelvey-Zavoina pseudo R2 for model
PseudoR2(model)


[1] "Logit Model for Whether or Not Participants Were Very Worried About Friends or Close Acquaintances Being Deported (Marginal Effect)"
[1] "N"


[1] 1012

factor,AME,SE,z,p,lower,upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
clinton,0.12001634,0.02840629,4.2249912,2.389501e-05,0.06434103,0.17569165
knowUndocumented,0.25280615,0.02512517,10.0618691,8.143733e-24,0.20356173,0.30205058
LanguageES,0.15673367,0.04484591,3.4949378,4.741722e-04,0.06883730,0.24463005
old,-0.19612250,0.03829408,-5.1214837,3.031410e-07,-0.27117752,-0.12106749
yearsInUS11+,-0.15988681,0.05542560,-2.8847108,3.917734e-03,-0.26851898,-0.05125464
yearsInUS6-10,-0.03016046,0.07715097,-0.3909278,6.958506e-01,-0.18137358,0.12105266


McFadden     Adj.McFadden        Cox.Snell       Nagelkerke 
       0.1257243        0.1139439        0.1552651        0.2101872 
McKelvey.Zavoina           Effron            Count        Adj.Count 
       0.2140431        0.1611297        0.6897233        0.2150000 
             AIC    Corrected.AIC 
    1201.4309506     1201.5425044

In [11]:
#Logit Model for Q17_3
print("Logit Models for Whether or Not Participants Were Very Worried About the Separation of Families due to deportation (Marginal Effect)")

#total observations for model 
print('N')
nrow(data[!is.na(data$Q17_3) & 
                 !is.na(data$Language) &
                 !is.na(data$knowUndocumented) & 
                 !is.na(data$Q11) &
                 !is.na(data$yearsInUS) &
                 !is.na(data$gender), ])

#model with Spanish, know undocumented, voted for Trump, voted for Clinton, male, and lived in US fewer than 5 years
model <-glm(veryWorriedFamilies ~ Language + knowUndocumented + clinton + trump + gender + yearsUS_5,
    family = binomial(link = "logit"),
    data = data)

#marginal effect
summary(margins(model))

#percent correctly predicted and Mckelvey-Zavoina pseudo R2 for model
PseudoR2(model)



[1] "Logit Models for Whether or Not Participants Were Very Worried About the Separation of Families due to deportation (Marginal Effect)"
[1] "N"


[1] 996

factor,AME,SE,z,p,lower,upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
clinton,0.12012393,0.03199954,3.753927,1.740857e-04,0.05740599,0.18284188
genderMale,-0.07313912,0.02976840,-2.456938,1.401269e-02,-0.13148412,-0.01479412
knowUndocumented,0.11544509,0.02800316,4.122573,3.746636e-05,0.06055990,0.17033028
LanguageES,0.11090936,0.04199307,2.641135,8.262876e-03,0.02860446,0.19321426
trump,-0.20409252,0.03734105,-5.465634,4.612561e-08,-0.27727964,-0.13090540
yearsUS_5,0.14389158,0.05947154,2.419503,1.554172e-02,0.02732951,0.26045365


McFadden     Adj.McFadden        Cox.Snell       Nagelkerke 
    8.653311e-02     7.427742e-02     1.072280e-01     1.468103e-01 
McKelvey.Zavoina           Effron            Count        Adj.Count 
    1.415983e-01     1.147236e-01     7.008032e-01     1.767956e-01 
             AIC    Corrected.AIC 
    1.206546e+03     1.206659e+03

In [12]:
#Logit Model for Q17_4
print("Logit Models for Whether or Not Participants Were Very Worried About Negative Impacts on the Economy from deportation (Marginal Effect)")

#total observations for model 
print('N')
nrow(data[!is.na(data$Q17_4) & 
                 !is.na(data$Language) &
                 !is.na(data$knowUndocumented) & 
                 !is.na(data$Q11) &
                 !is.na(data$age), ])

#model with Spanish, know Undocumented, voted for Trump, voted for Clinton, and 51+ years old
model <-glm(veryWorriedEconomy ~ Language + knowUndocumented + clinton + trump + old,
    family = binomial(link = "logit"),
    data = data)

#marginal effect
summary(margins(model))

#percent correctly predicted and Mckelvey-Zavoina pseudo R2 for model
PseudoR2(model)


[1] "Logit Models for Whether or Not Participants Were Very Worried About Negative Impacts on the Economy from deportation (Marginal Effect)"
[1] "N"


[1] 1009

factor,AME,SE,z,p,lower,upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
clinton,0.16447350,0.03183130,5.167037,2.378348e-07,0.10208530,0.22686171
knowUndocumented,0.07208605,0.02898383,2.487112,1.287848e-02,0.01527878,0.12889332
LanguageES,0.19058960,0.03765746,5.061138,4.167622e-07,0.11678233,0.26439686
old,-0.11534191,0.03565848,-3.234627,1.218016e-03,-0.18523126,-0.04545257
trump,-0.17385090,0.03857444,-4.506894,6.578356e-06,-0.24945542,-0.09824639


McFadden     Adj.McFadden        Cox.Snell       Nagelkerke 
    8.760311e-02     7.722564e-02     1.105297e-01     1.498953e-01 
McKelvey.Zavoina           Effron            Count        Adj.Count 
    1.447806e-01     1.169581e-01     6.798811e-01     1.781170e-01 
             AIC    Corrected.AIC 
    1.242893e+03     1.242977e+03

In [13]:
#Q19

print('For Participants Who Know an Undocumented Immigrant Living in the United States: To What Degree do Undocumented Immigrants You Know Well (Including Yourself) Worry About Participating in the Following Situations for Fear of Deportation?')

#function to split 7-point scale question into 3 categories and print out percentages for each category
Q19 <- function(x, word) {
    print(word)
    t <- table(as.factor(ifelse(x <= 3, 'do not worry', 
                         ifelse(x < 5 & x > 3, 'worry somewhat', 
                         ifelse(x >= 5, 'worry very much', NA)))))
    round(100*t/sum(t),1)
} 

#use function for each situation
Q19(data$Q19_1, 'Applying for scholarships/loans')
Q19(data$Q19_2, 'Health care and/or medical assistance')
Q19(data$Q19_3, 'Jobs')
Q19(data$Q19_4, 'Government welfare or assistance')
Q19(data$Q19_5, 'Reporting Crimes')
Q19(data$Q19_6, 'Attending public schools')



[1] "For Participants Who Know an Undocumented Immigrant Living in the United States: To What Degree do Undocumented Immigrants You Know Well (Including Yourself) Worry About Participating in the Following Situations for Fear of Deportation?"
[1] "Applying for scholarships/loans"



   do not worry  worry somewhat worry very much 
           21.7            15.5            62.8 

[1] "Health care and/or medical assistance"



   do not worry  worry somewhat worry very much 
           14.6            14.8            70.6 

[1] "Jobs"


...


   do not worry  worry somewhat worry very much 
           19.6            19.8            60.6 

[1] "Reporting Crimes"



   do not worry  worry somewhat worry very much 
           17.0            17.9            65.1 

[1] "Attending public schools"



   do not worry  worry somewhat worry very much 
           22.9            20.1            57.0 

In [14]:
#Chi-Squared table for Q20
print('Since the Election of Donald Trump, Do You Feel Levels of Discrimination Toward Latinos/Hispanics Have Increased?')

#use chi-squared function for each relevant characteristic and discrimination question
chiSquared(data$age, data$discrimination, 'Age')
chiSquared(data$gender, data$discrimination, 'Gender')
chiSquared(data$Language, data$discrimination, 'Language')
chiSquared(data$yearsInUS, data$discrimination, 'yearsInUS')
chiSquared(data$knowUndocumented, data$discrimination, 'knowUndocumented')

#discrimination question totals
print('Total')
print(100*table(data$discrimination)/sum(table(data$discrimination)))

#number of responses for discrimination question
print('N')
print(nrow(data[!is.na(data$discrimination),]))


[1] "Since the Election of Donald Trump, Do You Feel Levels of Discrimination Toward Latinos/Hispanics Have Increased?"
[1] "Age"
      not at all somewhat very much
18-30       14.8     16.1      69.1
31-50       14.3     14.3      71.5
51+         25.4     12.2      62.4



	Pearson's Chi-squared test

data:  t
X-squared = 14.933, df = 4, p-value = 0.004842


[1] "Gender"
       not at all somewhat very much
Female       13.3     14.5      72.2
Male         21.7     14.7      63.6



	Pearson's Chi-squared test

data:  t
X-squared = 13.065, df = 2, p-value = 0.001455


[1] "Language"
   not at all somewhat very much
EN       18.6     14.9      66.5
ES        5.7     12.1      82.3


...

[1] "yearsInUS"
     not at all somewhat very much
<5         10.5     15.1      74.4
11+        17.4     14.1      68.5
6-10       17.2     18.8      64.1



	Pearson's Chi-squared test

data:  t
X-squared = 3.745, df = 4, p-value = 0.4416


[1] "knowUndocumented"
  not at all somewhat very much
0       21.4     18.5      60.1
1       13.2     11.4      75.4



	Pearson's Chi-squared test

data:  t
X-squared = 27.146, df = 2, p-value = 1.275e-06


[1] "Total"

not at all   somewhat  very much 
  16.79764   14.53831   68.66405 
[1] "N"
[1] 1018


In [15]:
#Logit Model 1 for Q20 (has increased)
print("Logit Model 1 for Whether or Not Participants Felt That Discrimination Had Very Much Increased Since the Election of Donald Trump (Marginal Effect)")

#total observations for model 
print('N')
nrow(data[!is.na(data$discrimination) & 
                 !is.na(data$Language) &
                 !is.na(data$knowUndocumented) & 
                 !is.na(data$Q11), ])

#model with Spanish, know undocumented, voted for Trump, and voted for Clinton
model1 <-glm(discriminationIncreased ~ Language + knowUndocumented + clinton + trump,
    family = binomial(link = "logit"),
    data = data)

#marginal effect
summary(margins(model1))

#percent correctly predicted and Mckelvey-Zavoina pseudo R2 for model
PseudoR2(model1)



[1] "Logit Model 1 for Whether or Not Participants Felt That Discrimination Had Very Much Increased Since the Election of Donald Trump (Marginal Effect)"
[1] "N"


[1] 1012

factor,AME,SE,z,p,lower,upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
clinton,0.1535598,0.02971712,5.167386,2.373910e-07,0.09531535,0.2118043
knowUndocumented,0.1342353,0.02562465,5.238524,1.618660e-07,0.08401193,0.1844587
LanguageES,0.1519979,0.03427632,4.434488,9.229153e-06,0.08481755,0.2191782
trump,-0.2433414,0.03085262,-7.887219,3.089946e-15,-0.30381141,-0.1828714


McFadden     Adj.McFadden        Cox.Snell       Nagelkerke 
       0.1402896        0.1307644        0.1602445        0.2250542 
McKelvey.Zavoina           Effron            Count        Adj.Count 
       0.2182624        0.1772429        0.7480237        0.1981132 
             AIC    Corrected.AIC 
    1093.0841379     1093.1437800

In [16]:
#Logit Model 2 for Q20 (has increased)
print("Logit Model 2 for Whether or Not Participants Felt That Discrimination Had Very Much Increased Since the Election of Donald Trump (Marginal Effect)")

#total observations for model 
print('N')
nrow(data[!is.na(data$discrimination) & 
                 !is.na(data$Language) &
                 !is.na(data$knowUndocumented) & 
                 !is.na(data$Q11) & 
                 !is.na(data$age), ])

#model with Spanish, know undocumented, voted for Trump, voted for Clinton, Trump*old, and Clinton*old
model2 <-glm(discriminationIncreased ~ Language + knowUndocumented + clinton + trump + trump_old + clinton_old,
    family = binomial(link = "logit"),
    data = data)

#marginal effect
summary(margins(model2))

#percent correctly predicted and Mckelvey-Zavoina pseudo R2 for model
PseudoR2(model2)



[1] "Logit Model 2 for Whether or Not Participants Felt That Discrimination Had Very Much Increased Since the Election of Donald Trump (Marginal Effect)"
[1] "N"


[1] 1012

factor,AME,SE,z,p,lower,upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
clinton,0.1187759,0.03109693,3.819537,1.337024e-04,0.05782702,0.1797248
clinton_old,0.1452457,0.06089687,2.385110,1.707404e-02,0.02589003,0.2646014
knowUndocumented,0.1301478,0.02545246,5.113367,3.164661e-07,0.08026188,0.1800337
LanguageES,0.1377507,0.03394098,4.058535,4.938148e-05,0.07122757,0.2042738
trump,-0.1763312,0.03542721,-4.977283,6.448296e-07,-0.24576728,-0.1068952
trump_old,-0.2563298,0.07251642,-3.534783,4.081099e-04,-0.39845939,-0.1142002


McFadden     Adj.McFadden        Cox.Snell       Nagelkerke 
       0.1570413        0.1443412        0.1775754        0.2493945 
McKelvey.Zavoina           Effron            Count        Adj.Count 
       0.2573258        0.1952648        0.7559289        0.2232704 
             AIC    Corrected.AIC 
    1075.9798390     1076.0913928

In [17]:
#Logit Model 1 for Q20 (has not increased)
print("Logit Model 1 for Whether or Not Participants Felt That Discrimination Had Not at All Increased Since the Election of Donald Trump (Marginal Effect)")

#total observations for model 
print('N')
nrow(data[!is.na(data$discrimination) & 
                 !is.na(data$Language) &
                 !is.na(data$knowUndocumented) & 
                 !is.na(data$Q11) &
                 !is.na(data$gender), ])

#model with Spanish, know undocumented, voted for Trump, voted for Clinton, and male
model1 <-glm(discriminationNotIncreased ~ Language + knowUndocumented + clinton + trump + gender,
    family = binomial(link = "logit"),
    data = data)

#marginal effect
summary(margins(model1))

#percent correctly predicted and Mckelvey-Zavoina pseudo R2 for model
PseudoR2(model1)


[1] "Logit Model 1 for Whether or Not Participants Felt That Discrimination Had Not at All Increased Since the Election of Donald Trump (Marginal Effect)"
[1] "N"


[1] 1008

factor,AME,SE,z,p,lower,upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
clinton,-0.12239165,0.02850307,-4.293981,1.754974e-05,-0.178256650,-0.06652666
genderMale,0.04711786,0.02115140,2.227647,2.590405e-02,0.005661874,0.08857384
knowUndocumented,-0.06364044,0.02036834,-3.124478,1.781211e-03,-0.103561658,-0.02371921
LanguageES,-0.12450129,0.02365897,-5.262328,1.422425e-07,-0.170872027,-0.07813055
trump,0.19622887,0.01924923,10.194118,2.106484e-24,0.158501079,0.23395666


McFadden     Adj.McFadden        Cox.Snell       Nagelkerke 
       0.2334719        0.2181670        0.1909312        0.3201070 
McKelvey.Zavoina           Effron            Count        Adj.Count 
       0.3341083        0.2503082        0.8581349        0.1588235 
             AIC    Corrected.AIC 
     713.1744311      713.2583472

In [18]:
#Logit Model 2 for Q20 (has not increased)
print("Logit Model 2 for Whether or Not Participants Felt That Discrimination Had Not at All Increased Since the Election of Donald Trump (Marginal Effect)")

#total observations for model 
print('N')
nrow(data[!is.na(data$discrimination) & 
                 !is.na(data$Language) &
                 !is.na(data$knowUndocumented) & 
                 !is.na(data$Q11) &
                 !is.na(data$gender) &
                 !is.na(data$age), ])

#model with Spanish, know undocumented, voted for Trump, voted for Clinton, male, and trump*old
model2 <-glm(discriminationNotIncreased ~ Language + knowUndocumented + clinton + trump + gender + trump_old,
    family = binomial(link = "logit"),
    data = data)

#marginal effect
summary(margins(model2))

#percent correctly predicted and Mckelvey-Zavoina pseudo R2 for model
PseudoR2(model2)

[1] "Logit Model 2 for Whether or Not Participants Felt That Discrimination Had Not at All Increased Since the Election of Donald Trump (Marginal Effect)"
[1] "N"


[1] 1008

factor,AME,SE,z,p,lower,upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
clinton,-0.11888146,0.02792587,-4.257037,2.071539e-05,-0.173615150,-0.06414776
genderMale,0.04492560,0.02094434,2.144999,3.195289e-02,0.003875447,0.08597576
knowUndocumented,-0.05584481,0.02027350,-2.754572,5.876895e-03,-0.095580129,-0.01610948
LanguageES,-0.11546515,0.02483034,-4.650163,3.316722e-06,-0.164131724,-0.06679857
trump,0.15951219,0.02308401,6.910073,4.844040e-12,0.114268363,0.20475602
trump_old,0.10545507,0.03504962,3.008736,2.623367e-03,0.036759074,0.17415107


McFadden     Adj.McFadden        Cox.Snell       Nagelkerke 
       0.2434225        0.2259312        0.1982041        0.3323006 
McKelvey.Zavoina           Effron            Count        Adj.Count 
       0.3351378        0.2655096        0.8640873        0.1941176 
             AIC    Corrected.AIC 
     706.0722028      706.1842028